In [1]:
import os
import time
import random
import warnings
import typing as tp
import pathlib
from contextlib import contextmanager

import cv2
import librosa

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import IPython.display as ipd

import torch.nn as nn
import torch.optim as optim

from torchvision import datasets, models, transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau


In [2]:
import python_speech_features as psf
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
data_path = pathlib.Path("../data")
audios_path = data_path / "all_audio_resampled"

In [4]:
train = pd.read_csv(data_path/'Train.csv')
train_extra = pd.read_csv(data_path/'train_add.csv')
train_extra_2 = pd.read_csv(data_path/'train_add_20201029.csv')

label2code = {word: idx for idx, word in enumerate(train.label.unique().tolist())}
code2label = {v:k for k,v in label2code.items()}

In [5]:
def get_image_path(audio_path):
    file_name = audio_path.split("/")[-1].split(".")[0]
    ip = str(audios_path.resolve() / f"{file_name}.wav")
    return ip

train["image_fn"] = train.fn.apply(get_image_path)
train_extra["image_fn"] = train_extra.fn.apply(get_image_path)
train_extra_2["image_fn"] = train_extra_2.fn.apply(get_image_path)

In [6]:
train_df = pd.concat([train, train_extra, train_extra_2], axis=0).reset_index(drop=True)

In [7]:
### build validation that includes all classes:


vcs = train_df.label.value_counts()

## possible schema:
# 25+ - take 3
# 12-25 - take 2
# 12- - take 1

def num_for_val(num_examples):
    if num_examples >= 25:
        return 3
    if num_examples >= 12:
        return 2
    return 1

train_df["num_examples"] = train_df.label.map(vcs.to_dict())
train_df["num_for_val"] = train_df.num_examples.apply(num_for_val)

random.seed(12)
train_df_new = pd.DataFrame()
for label in train_df.label.unique():
    tmp = train_df.loc[train_df.label == label].copy()
    tmp["dummy"] = tmp.label.apply(lambda _: random.random())
    tmp.sort_values(by="dummy", inplace=True)
    tmp["rank"] = range(tmp.shape[0])
    train_df_new = pd.concat([train_df_new, tmp])

train_df_new.reset_index(drop=True, inplace=True)
train_df_new["val_subset"] = train_df_new.num_for_val > train_df_new["rank"]
train_df_new.drop("dummy", axis=1, inplace=True)

In [8]:
assert (
    set(train_df_new.loc[train_df_new.val_subset].label.unique()) == 
    set(train_df_new.loc[~train_df_new.val_subset].label.unique())
)

In [9]:
train_df_new["val_fold"] = train_df_new["rank"] // train_df_new.num_for_val

In [10]:
for f in range(10):
    print(f, (train_df_new.val_fold == f).sum())

0 451
1 451
2 451
3 430
4 413
5 410
6 407
7 391
8 381
9 337


In [11]:
N_FOLDS = 3
train_folds = list()
val_folds = list()

for i in range(N_FOLDS):
    valf = train_df_new.loc[train_df_new.val_fold == i].copy()
    trf = train_df_new.loc[train_df_new.val_fold != i].copy()
    
    train_folds.append(trf)
    val_folds.append(valf)

In [12]:
SR = 22050

In [13]:
from reproduce.src.data_processing import new_generate_spec, new_build_image, normalize, MEAN, STD

In [14]:
class SpectrogramDataset(data.Dataset):
    def __init__(
        self,
        file_list: tp.List[tp.List[str]],
        config,
        transform=None,
        normalize=True
    ):
        self.file_list = file_list  # list of list: [file_path, label]
        self.transform = transform
        self.normalize = normalize
        self.config = config

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx: int):
        fn, word = self.file_list[idx]
        audio, _ = librosa.core.load(fn, sr=SR)

        if self.transform:
            audio = self.transform(audio)

        image = new_build_image(audio, self.config)
        
        if self.normalize:
            norm_image = normalize(np.array(image), mean=MEAN, std=STD)
        else:
            norm_image = image
        
        return np.moveaxis(norm_image, 2, 0), label2code[word]
    
    
class SpectrogramTestDataset(data.Dataset):
    def __init__(
        self,
        file_list: tp.List[tp.List[str]],
        config,
        transform=None,
        normalize=True
    ):
        self.file_list = file_list  # list of list: [file_path, label]
        self.transform = transform
        self.normalize = normalize
        self.config = config

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx: int):
        fn, word = self.file_list[idx]
        audio, _ = librosa.core.load(fn, sr=SR)

        if self.transform:
            audio = self.transform(audio)

        image = new_build_image(audio, self.config)
        
        if self.normalize:
            norm_image = normalize(np.array(image), mean=MEAN, std=STD)
        else:
            norm_image = image
        
        return np.moveaxis(norm_image, 2, 0)

In [15]:
class AudioConfig:
    n_fft = 512
    hop_size = 32
    pad_center = True
    trim = True
    max_len_sec = 2.6
    sr = 22050
    img_size = 299
    
conf = AudioConfig()

In [16]:
sdf_train_list = [
    SpectrogramDataset(t[["image_fn", "label"]].values.tolist(), conf, normalize=True)
    for t in train_folds
]

sdf_val_list = [
    SpectrogramDataset(v[["image_fn", "label"]].values.tolist(), conf, normalize=True)
    for v in val_folds
]

for s in sdf_train_list:
    print(len(s))
    
print("=========================")

for s in sdf_val_list:
    print(len(s))

4258
4258
4258
451
451
451


In [17]:
def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [18]:
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def train_mixup_epoch(log_interval, mixup_prob, model, device, criterion, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        use_mixup = False
        if random.random() < mixup_prob:
            use_mixup = True
        data, target = data.type(torch.FloatTensor).to(device), target.to(device)
        
        if use_mixup:
            data, y_a, y_b, lam = mixup_data(data, target)
        optimizer.zero_grad()
        output = model(data)
        if use_mixup:
            loss = mixup_criterion(criterion, output, y_a, y_b, lam) #criterion(output, target)
        else:
            loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    return loss.item()

            
def test(model, device, criterion, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.type(torch.FloatTensor).to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    
    accuracy = 100. * correct / len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        accuracy))

    return test_loss, accuracy

In [19]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)


def train_on_fold(
    model,
    train_loader,
    val_loader,
    fold,
    criterion,
    optimizer,
    lr_scheduler,
    results_folder,
    epochs
):
    seed_dict = {
        0: 9,
        1: 99,
        2: 999
    }
    set_seed(seed_dict[fold])

    t0 = time.time()

    best_loss = 1e5
    best_acc = 0

    max_patience = 20
    patience = 0

    train_loss_hist = list()
    val_loss_hist = list()
    val_acc_hist = list()

    save_each_epoch = False

    for ep in range(55):
        train_loss = train_mixup_epoch(1e10, 0.667, model, device, criterion, train_loader, optimizer, ep)
        cur_loss, cur_acc = test(model, device, criterion, val_loader)

        train_loss_hist.append(train_loss)
        val_loss_hist.append(cur_loss)
        val_acc_hist.append(cur_acc)

        if save_each_epoch:
            torch.save(model.state_dict(), f"tmp/{results_folder}/model_ep_{fold}_{ep}.pth")

        if cur_loss < best_loss:
            torch.save(model.state_dict(), f"tmp/{results_folder}/best_run_{fold}.pth")
            best_loss = cur_loss
            best_acc = cur_acc
            patience = 0

        lr_scheduler.step(cur_loss) 
        print("Training so far {} minutes".format((time.time() - t0) / 60))
        print("="*20)
        
    for ep in range(11):
        train_loss = train_mixup_epoch(1e10, 0.0, model, device, criterion, train_loader, optimizer, ep)
        cur_loss, cur_acc = test(model, device, criterion, val_loader)

        train_loss_hist.append(train_loss)
        val_loss_hist.append(cur_loss)
        val_acc_hist.append(cur_acc)

        if save_each_epoch:
            torch.save(model.state_dict(), f"tmp/{results_folder}/model_ep_{fold}_{ep}.pth")

        if cur_loss < best_loss:
            torch.save(model.state_dict(), f"tmp/{results_folder}/best_run_{fold}.pth")
            best_loss = cur_loss
            best_acc = cur_acc
            patience = 0

        lr_scheduler.step(cur_loss)
        print("Training so far {} minutes".format((time.time() - t0) / 60))
        print("="*20)

    print("time spent training: {} minutes".format((time.time() - t0) / 60))
    print("BEST LOSS:", best_loss)
    print("BEST ACC:", best_acc)
    
    model.load_state_dict(torch.load(f"tmp/{results_folder}/best_run_{fold}.pth"))
    
    return model


def predict_on_val(
    model,
    val_loader,
    device="cuda"
):
    sfm = nn.Softmax()
    predictions = list()
    for batch_idx, (inputs, _) in enumerate(val_loader):
        inputs = inputs.type(torch.FloatTensor).to(device)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            predictions.append(sfm(outputs))

    predictions = np.concatenate([t.cpu().numpy() for t in predictions])
    preds = np.argmax(predictions, axis=1)
    return preds


def get_val_outputs(model, test_loader, device="cuda"):
    outputs_list = list()
    for batch_idx, (inputs, target) in enumerate(test_loader):
        inputs = inputs.type(torch.FloatTensor).to(device)

        # forward
        # track history if only in train
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            outputs_list.append(outputs)
    outputs_list = np.concatenate([t.cpu().numpy() for t in outputs_list])
    return outputs_list

def get_predictions(model, test_loader, device="cuda"):
    sfm = nn.Softmax()
    predictions = list()
    for batch_idx, inputs in enumerate(test_loader):
        inputs = inputs.type(torch.FloatTensor).to(device)

        # forward
        # track history if only in train
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            predictions.append(sfm(outputs)) ## ADD SOFTMAX
    predictions = np.concatenate([t.cpu().numpy() for t in predictions])
    return predictions

In [20]:
# model_ft.load_state_dict(torch.load(f"tmp/{tmp_folder_name}/best_run_{FOLD}.pth"))

In [21]:
n_classes = len(label2code)
device = torch.device("cuda")

results_folder = "densenet161_mixup_repr_folds"
os.mkdir(f"tmp/{results_folder}")

for FOLD in range(N_FOLDS):
    print("PROCESSING FOLD", FOLD)
    model_ft = models.densenet161(pretrained=True)
    model_ft.classifier = nn.Sequential(
        nn.Linear(model_ft.classifier.in_features, n_classes)
    )
    model_ft = model_ft.to(device)

    criterion = nn.CrossEntropyLoss(reduction="sum")
    optimizer_ft = optim.Adam(model_ft.parameters(), lr=5e-4)
    lr_scheduler = ReduceLROnPlateau(optimizer_ft, 'min', patience=4, factor=0.5, verbose=True, min_lr=1.5e-5)

    train_loader = torch.utils.data.DataLoader(sdf_train_list[FOLD], batch_size=16, shuffle=True,
                                               num_workers=6, drop_last=False, pin_memory=True)
    val_loader = torch.utils.data.DataLoader(sdf_val_list[FOLD], batch_size=16, drop_last=False, pin_memory=True,
                                        num_workers=6)
    
    # full train cycle with one fold for cross-val:
    model_ft = train_on_fold(
        model_ft,
        train_loader,
        val_loader,
        FOLD,
        criterion,
        optimizer_ft,
        lr_scheduler,
        results_folder,
        None
    )
    
    # predict from one model:
    preds = predict_on_val(
        model_ft,
        val_loader
    )
    val_folds[FOLD]["preds"] = [code2label[c] for c in preds]
    
    # Save val predictions from one model:
    val_out = get_val_outputs(model_ft, val_loader)
    gt = val_folds[FOLD].label.map(label2code).values
    val_losses = list()
    
    for idx in range(len(gt)):
        item_loss = criterion(torch.Tensor([val_out[idx]]), torch.LongTensor([gt[idx]])).numpy()
        val_losses.append(item_loss)
    val_losses = np.array(val_losses)
    val_folds[FOLD]["loss"] = val_losses
    val_folds[FOLD].reset_index(drop=True).sort_values(by="loss", ascending=True).to_csv(
        f"tmp/{results_folder}/val_loss_{FOLD}.csv", index=False
    )
    
    ## for debug, save predictions for train as well:
    ### debug train audios:
    train_debug_loader = torch.utils.data.DataLoader(sdf_train_list[FOLD], batch_size=32, shuffle=False,
                                               num_workers=4, drop_last=False, pin_memory=True)
    train_out = get_val_outputs(model_ft, train_debug_loader)
    gt = train_folds[FOLD].label.map(label2code).values
    train_losses = list()

    for idx in range(len(gt)):
        item_loss = criterion(torch.Tensor([train_out[idx]]), torch.LongTensor([gt[idx]])).numpy()
        train_losses.append(item_loss)

    train_losses = np.array(train_losses)
    train_folds[FOLD]["loss"] = train_losses
    train_folds[FOLD]["preds"] = [code2label[c] for c in np.argmax(train_out, axis=1)]
    train_folds[FOLD].reset_index(drop=True).sort_values(by="loss", ascending=False).to_csv(
        f"tmp/{results_folder}/train_loss_{FOLD}.csv", index=False
    )
    
    ## MAKE SUBMISSION (from one model):
    sample_subm = pd.read_csv("Submission1.csv")
    sample_subm["image_fn"] = sample_subm.fn.apply(get_image_path)
    subm_dataset = SpectrogramTestDataset([[path, None] for path in sample_subm.image_fn.values ], conf)
    subm_loader = torch.utils.data.DataLoader(subm_dataset, batch_size=16)

    preds = get_predictions(model_ft, subm_loader)

    for c in sample_subm.columns:
        if c in {"fn", "image_fn"}:
            continue
        c_idx = label2code[c]
        sample_subm[c] = preds[:, c_idx]

    sample_subm.drop("image_fn", axis=1).to_csv(f'tmp/{results_folder}/subm_{FOLD}.csv', index=False)

PROCESSING FOLD 0
Train Epoch: 0 [0/4258 (0%)]	Loss: 85.442596

Test set: Average loss: 8.8587, Accuracy: 2/451 (0%)

Training so far 1.9941056887308757 minutes
Train Epoch: 1 [0/4258 (0%)]	Loss: 79.447342

Test set: Average loss: 4.9973, Accuracy: 8/451 (2%)

Training so far 4.0111941734949745 minutes
Train Epoch: 2 [0/4258 (0%)]	Loss: 79.939110

Test set: Average loss: 5.2344, Accuracy: 11/451 (2%)

Training so far 6.010459589958191 minutes
Train Epoch: 3 [0/4258 (0%)]	Loss: 71.284996

Test set: Average loss: 7.5468, Accuracy: 9/451 (2%)

Training so far 8.022417342662811 minutes
Train Epoch: 4 [0/4258 (0%)]	Loss: 71.251129

Test set: Average loss: 4.4163, Accuracy: 20/451 (4%)

Training so far 10.032903158664704 minutes
Train Epoch: 5 [0/4258 (0%)]	Loss: 61.629967

Test set: Average loss: 4.4396, Accuracy: 31/451 (7%)

Training so far 12.037156709035237 minutes
Train Epoch: 6 [0/4258 (0%)]	Loss: 75.835556

Test set: Average loss: 3.8339, Accuracy: 57/451 (13%)

Training so far 14.04

Train Epoch: 49 [0/4258 (0%)]	Loss: 30.209393

Test set: Average loss: 0.6320, Accuracy: 392/451 (87%)

Training so far 100.01761779387792 minutes
Train Epoch: 50 [0/4258 (0%)]	Loss: 0.080884

Test set: Average loss: 0.6989, Accuracy: 380/451 (84%)

Training so far 102.0129777987798 minutes
Train Epoch: 51 [0/4258 (0%)]	Loss: 29.511700

Test set: Average loss: 0.6338, Accuracy: 391/451 (87%)

Training so far 104.00855114062627 minutes
Train Epoch: 52 [0/4258 (0%)]	Loss: 0.140758

Test set: Average loss: 0.6974, Accuracy: 384/451 (85%)

Training so far 106.0031378587087 minutes
Train Epoch: 53 [0/4258 (0%)]	Loss: 4.511357

Test set: Average loss: 0.6706, Accuracy: 384/451 (85%)

Epoch    54: reducing learning rate of group 0 to 1.2500e-04.
Training so far 107.99689534902572 minutes
Train Epoch: 54 [0/4258 (0%)]	Loss: 19.236303

Test set: Average loss: 0.6447, Accuracy: 393/451 (87%)

Training so far 110.00432965358098 minutes
Train Epoch: 0 [0/4258 (0%)]	Loss: 0.082168

Test set: Averag

/home/letfoolsdie/virtual_envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:104: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/letfoolsdie/virtual_envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


PROCESSING FOLD 1
Train Epoch: 0 [0/4258 (0%)]	Loss: 85.458000

Test set: Average loss: 5.1605, Accuracy: 3/451 (1%)

Training so far 1.989265294869741 minutes
Train Epoch: 1 [0/4258 (0%)]	Loss: 88.092468

Test set: Average loss: 5.0036, Accuracy: 7/451 (2%)

Training so far 3.983501180013021 minutes
Train Epoch: 2 [0/4258 (0%)]	Loss: 77.267563

Test set: Average loss: 5.2043, Accuracy: 8/451 (2%)

Training so far 5.98129030863444 minutes
Train Epoch: 3 [0/4258 (0%)]	Loss: 77.231834

Test set: Average loss: 4.5449, Accuracy: 19/451 (4%)

Training so far 7.976518352826436 minutes
Train Epoch: 4 [0/4258 (0%)]	Loss: 66.879356

Test set: Average loss: 4.2600, Accuracy: 38/451 (8%)

Training so far 9.96908281246821 minutes
Train Epoch: 5 [0/4258 (0%)]	Loss: 61.727505

Test set: Average loss: 3.5773, Accuracy: 69/451 (15%)

Training so far 11.964548973242442 minutes
Train Epoch: 6 [0/4258 (0%)]	Loss: 50.228035

Test set: Average loss: 2.6071, Accuracy: 174/451 (39%)

Training so far 13.95784

Train Epoch: 48 [0/4258 (0%)]	Loss: 7.684467

Test set: Average loss: 0.7125, Accuracy: 384/451 (85%)

Training so far 97.99973359107972 minutes
Train Epoch: 49 [0/4258 (0%)]	Loss: 8.049229

Test set: Average loss: 0.7292, Accuracy: 387/451 (86%)

Training so far 99.98929142951965 minutes
Train Epoch: 50 [0/4258 (0%)]	Loss: 0.089929

Test set: Average loss: 0.7388, Accuracy: 385/451 (85%)

Training so far 102.00115214188894 minutes
Train Epoch: 51 [0/4258 (0%)]	Loss: 36.451164

Test set: Average loss: 0.7188, Accuracy: 387/451 (86%)

Training so far 103.98676419655482 minutes
Train Epoch: 52 [0/4258 (0%)]	Loss: 12.203717

Test set: Average loss: 0.7671, Accuracy: 384/451 (85%)

Training so far 105.97477976878484 minutes
Train Epoch: 53 [0/4258 (0%)]	Loss: 0.108616

Test set: Average loss: 0.7247, Accuracy: 384/451 (85%)

Epoch    54: reducing learning rate of group 0 to 1.5625e-05.
Training so far 107.96411498387654 minutes
Train Epoch: 54 [0/4258 (0%)]	Loss: 0.078841

Test set: Averag

Train Epoch: 29 [0/4258 (0%)]	Loss: 0.349782

Test set: Average loss: 0.8514, Accuracy: 381/451 (84%)

Training so far 60.811685132980344 minutes
Train Epoch: 30 [0/4258 (0%)]	Loss: 10.479640

Test set: Average loss: 0.8403, Accuracy: 375/451 (83%)

Training so far 62.813581871986386 minutes
Train Epoch: 31 [0/4258 (0%)]	Loss: 31.000462

Test set: Average loss: 0.8724, Accuracy: 378/451 (84%)

Training so far 64.81982274850209 minutes
Train Epoch: 32 [0/4258 (0%)]	Loss: 0.708453

Test set: Average loss: 0.9101, Accuracy: 372/451 (82%)

Training so far 66.82838279803595 minutes
Train Epoch: 33 [0/4258 (0%)]	Loss: 25.683498

Test set: Average loss: 0.8119, Accuracy: 383/451 (85%)

Training so far 68.8321104447047 minutes
Train Epoch: 34 [0/4258 (0%)]	Loss: 33.790890

Test set: Average loss: 0.8609, Accuracy: 374/451 (83%)

Training so far 70.84204442501068 minutes
Train Epoch: 35 [0/4258 (0%)]	Loss: 34.530617

Test set: Average loss: 0.8436, Accuracy: 376/451 (83%)

Training so far 72.89

In [26]:
[print(i/100) for i in [87.13968957871397,86.69623059866963,84.70066518847007]]

0.8713968957871397
0.8669623059866963
0.8470066518847007


[None, None, None]

In [23]:
## MAKE ONE SUBMISSION FROM ALL SUBS:
import pandas as pd
import numpy as np

s = f'tmp/{results_folder}/'
preds_to_average = [
    f"subm_{f}.csv"
    for f in range(N_FOLDS)
]

all_subs = list()
source_pred = pd.read_csv(s + preds_to_average[0])
pred_cols = source_pred.drop("fn", axis=1).columns.values
for file in preds_to_average[1:]:
    tmp = pd.read_csv(s + file)
    source_pred[pred_cols] += tmp[pred_cols]

source_pred[pred_cols] /= len(preds_to_average)

In [24]:
np.all(np.isclose((source_pred[pred_cols].sum(axis=1)).values, np.ones(source_pred.shape[0])))

True

In [25]:
source_pred.to_csv(f'tmp/{results_folder}/{results_folder}_merged.csv', index=False)